#2.1

In [1]:
import numpy as np

In [2]:
def read_pgm(pgmf):
    rs = pgmf.readline()
    rd = pgmf.readline()
    if str(rd).find('#') != -1 :
        rd = pgmf.readline()
    (width, height) = [int(i) for i in rd.split()]

    raster = []
    for y in range(height):
        row = []
        for x in range(width):
            row.append(ord(pgmf.read(1)))
        raster.append(row)
    return raster

In [3]:
def writePGM(file, w, h, max_grayscale, flatImg):
    file.write('P5\n'.encode())
    file.write('{} {}\n'.format(w, h).encode())
    file.write('{}\n'.format(max_grayscale).encode())
    file.write(bytearray(flatImg))
    file.close()

In [4]:
def padding_center(img, pad):
    pading = [255]*(pad//2)
    l = len(img[0])
    for row in img :
        for zero in pading :
            row.insert(0, zero)
        row.extend(pading)
    pading = [[255]*(l + 2 * (pad//2)) for _ in range(pad//2)]
    for row in pading:
        img.insert(0, row)
    img.extend(pading)

In [5]:
def contrast_s(li):
    result = []
    c = np.amin(li)
    d = np.amax(li)
    a = 0
    b = 255
    for i in range(len(li)):
        row = []
        for j in range(len(li[0])):
            p = li[i][j]
            p_out = (p - c)*((b - a)/(d - c)) + a
            row.append(int(p_out))
        result.append(row)
    return result

In [6]:
def flatten(arr):
    return sum(arr, [])

In [7]:
a = open("D:\Project\DIP\HW2\Picture\Cross.pgm",'rb')
img = read_pgm(a)
a.close()

In [8]:
padding_center(img,56)
img[28][28] = 255
img[28][29] = 255
img[28][30] = 255
img[28][31] = 255

In [10]:
def ideal_lowpass_filter(img,cutoff):
    m = len(img)
    n = len(img[0])
    list_filter = [[255]*m for _ in range(n)]
    for i in range(m):
        for j in range(n):
            d = float(np.sqrt(((float(j)-m/2.0)**2.0)+((float(i)-n/2)**2.0)))
            if d <= float(cutoff):
                list_filter[i][j] = 1.0
            else :
                list_filter[i][j] = 0.0
    fft_img = np.fft.fftshift(np.fft.fft2(img))
    return_list = fft_img * list_filter
    invert = np.fft.ifft2(np.fft.ifftshift(return_list))
    abs_invert = np.abs(invert)
    return abs_invert

In [11]:
def gaussian_lowpass_filter(img,cutoff):
    m = len(img)
    n = len(img[0])
    list_filter = [[255]*m for _ in range(n)]
    for i in range(m):
        for j in range(n):
            d = float(np.sqrt(((float(j)-m/2.0)**2.0)+((float(i)-n/2)**2.0)))
            list_filter[i][j] = np.exp(-1*((d**2.0)/((2.0)*(cutoff**2.0))))
    fft_img = np.fft.fftshift(np.fft.fft2(img))
    return_list = fft_img * list_filter
    invert = np.fft.ifft2(np.fft.ifftshift(return_list))
    abs_invert = np.abs(invert)
    return abs_invert

In [12]:
ideal_20 = ideal_lowpass_filter(img,20)
ideal_30 = ideal_lowpass_filter(img,30)
ideal_50 = ideal_lowpass_filter(img,50)

In [13]:
ideal_contrast_20 = contrast_s(ideal_20)
ideal_contrast_30 = contrast_s(ideal_30)
ideal_contrast_50 = contrast_s(ideal_50)

In [49]:
gaussian_20 = gaussian_lowpass_filter(img,20)
gaussian_30 = gaussian_lowpass_filter(img,30)
gaussian_50 = gaussian_lowpass_filter(img,50)

In [50]:
gaussian_contrast_20 = contrast_s(gaussian_20)
gaussian_contrast_30 = contrast_s(gaussian_30)
gaussian_contrast_50 = contrast_s(gaussian_50)

Part save

In [40]:
save_dir = open("./Picture/Cross_ideal_lowpass_20.pgm","wb")
writePGM(save_dir , len(ideal_contrast_20[0]) , len(ideal_contrast_20) , 255 , flatten(ideal_contrast_20))

In [38]:
save_dir = open("./Picture/Cross_ideal_lowpass_30.pgm","wb")
writePGM(save_dir , len(ideal_contrast_30[0]) , len(ideal_contrast_30) , 255 , flatten(ideal_contrast_30))

In [39]:
save_dir = open("./Picture/Cross_ideal_lowpass_50.pgm","wb")
writePGM(save_dir , len(ideal_contrast_50[0]) , len(ideal_contrast_50) , 255 , flatten(ideal_contrast_50))

In [51]:
save_dir = open("./Picture/Cross_Gaussian_lowpass_20.pgm","wb")
writePGM(save_dir , len(gaussian_contrast_20[0]) , len(gaussian_contrast_20) , 255 , flatten(gaussian_contrast_20))

In [52]:
save_dir = open("./Picture/Cross_Gaussian_lowpass_30.pgm","wb")
writePGM(save_dir , len(gaussian_contrast_30[0]) , len(gaussian_contrast_30) , 255 , flatten(gaussian_contrast_30))

In [53]:
save_dir = open("./Picture/Cross_Gaussian_lowpass_50.pgm","wb")
writePGM(save_dir , len(gaussian_contrast_50[0]) , len(gaussian_contrast_50) , 255 , flatten(gaussian_contrast_50))

#2.2

In [7]:
def convolution(img,kernel):
    n = len(img)
    result_img = [[255]*n for _ in img]
    for i in range(1,n-1):
        for j in range(1,n-1):
            temp = 0
            temp += img[i][j] * kernel[1][1]
            temp += img[i-1][j-1] * kernel[0][0]
            temp += img[i-1][j] * kernel[0][1]
            temp += img[i+1][j+1] * kernel[2][2]
            temp += img[i][j-1] * kernel[1][0]
            temp += img[i][j+1] * kernel[1][2]
            temp += img[i+1][j-1] * kernel[2][0]
            temp += img[i+1][j] * kernel[2][1]
            temp += img[i+1][j+1] * kernel[2][2]
            result_img[i][j] = int(temp)
    return result_img

In [8]:
def median_filter(img):
    cp_img = img.copy()
    padding_center(cp_img,2)
    kernel = [[1/9, 1/9, 1/9],[1/9, 1/9, 1/9],[1/9, 1/9, 1/9]]
    result = convolution(cp_img,kernel)
    return result

In [9]:
def RMS(ori_img,aff_img):
    m = len(ori_img)
    n = len(ori_img[0])
    sigma = 0.0
    for i in range(m):
        for j in range(n):
            sigma = sigma + ((ori_img[i][j]-aff_img[i][j])**2)
    root = (sigma/(m*n))
    sqrt = np.sqrt(root)
    return sqrt

In [10]:
l = open(".\Picture\Lenna_noise.pgm",'rb')
lenna_noise_img = read_pgm(l)
l.close()

In [129]:
c = open(".\Picture\Chess_noise.pgm",'rb')
chess_noise_img = read_pgm(c)
c.close()

Lenna

In [21]:
lenna_ideal_20 = ideal_lowpass_filter(lenna_noise_img,20)
lenna_ideal_30 = ideal_lowpass_filter(lenna_noise_img,30)
lenna_ideal_50 = ideal_lowpass_filter(lenna_noise_img,50)

In [22]:
lenna_ideal_contrast_20 = contrast_s(lenna_ideal_20)
lenna_ideal_contrast_30 = contrast_s(lenna_ideal_30)
lenna_ideal_contrast_50 = contrast_s(lenna_ideal_50)

In [54]:
lenna_gaussian_20 = gaussian_lowpass_filter(lenna_noise_img,20)
lenna_gaussian_30 = gaussian_lowpass_filter(lenna_noise_img,30)
lenna_gaussian_50 = gaussian_lowpass_filter(lenna_noise_img,50)

In [55]:
lenna_gaussian_contrast_20 = contrast_s(lenna_gaussian_20)
lenna_gaussian_contrast_30 = contrast_s(lenna_gaussian_30)
lenna_gaussian_contrast_50 = contrast_s(lenna_gaussian_50)

In [11]:
lenna_median_filter_ = median_filter(lenna_noise_img)

In [12]:
lenna_contrast = contrast_s(lenna_median_filter_)

Chess

In [27]:
chess_ideal_20 = ideal_lowpass_filter(chess_noise_img,20)
chess_ideal_30 = ideal_lowpass_filter(chess_noise_img,30)
chess_ideal_50 = ideal_lowpass_filter(chess_noise_img,50)

In [28]:
chess_ideal_contrast_20 = contrast_s(chess_ideal_20)
chess_ideal_contrast_30 = contrast_s(chess_ideal_30)
chess_ideal_contrast_50 = contrast_s(chess_ideal_50)

In [56]:
chess_gaussian_20 = gaussian_lowpass_filter(chess_noise_img,20)
chess_gaussian_30 = gaussian_lowpass_filter(chess_noise_img,30)
chess_gaussian_50 = gaussian_lowpass_filter(chess_noise_img,50)

In [57]:
chess_gaussian_contrast_20 = contrast_s(chess_gaussian_20)
chess_gaussian_contrast_30 = contrast_s(chess_gaussian_30)
chess_gaussian_contrast_50 = contrast_s(chess_gaussian_50)

In [98]:
chess_median_filter_non_pad = median_filter(chess_noise_img)

In [99]:
chess_contrast = contrast_s(chess_median_filter_non_pad)

Find RMS 

lenna

In [14]:
l_ori = open(".\Picture\Lenna.pgm",'rb')
l_noi = open(".\Picture\Lenna_noise.pgm",'rb')
lenna_original = read_pgm(l_ori)
lenna_noise = read_pgm(l_noi)
l_ori.close()
l_noi.close()


In [102]:
print("original :",RMS(lenna_original,lenna_original))
print("noise :",RMS(lenna_original,lenna_noise_img))
print("ideal 20 :",RMS(lenna_original,lenna_ideal_20))
print("ideal 30 :",RMS(lenna_original,lenna_ideal_30))
print("ideal 50 :",RMS(lenna_original,lenna_ideal_50))


original : 0.0
noise : 25.280217586981273
ideal 20 : 17.61927403334903
ideal 30 : 14.214403908538953
ideal 50 : 12.700397107688488


In [92]:
print("original :",RMS(lenna_original,lenna_original))
print("noise :",RMS(lenna_original,lenna_noise))
print("gaussian 20 :",RMS(lenna_original,lenna_gaussian_20))
print("gaussian 30 :",RMS(lenna_original,lenna_gaussian_30))
print("gaussian 50 :",RMS(lenna_original,lenna_gaussian_50))

original : 0.0
noise : 25.280217586981273
gaussian 20 : 14.475296833575943
gaussian 30 : 11.945982237369638
gaussian 50 : 11.22485552770879


In [15]:
print("original :",RMS(lenna_original,lenna_original))
print("noise :",RMS(lenna_original,lenna_noise))
print("median :",RMS(lenna_original,lenna_median_filter_))

original : 0.0
noise : 25.280217586981273
median : 21.271354447226233


chess

In [109]:
c_ori = open(".\Picture\Chess.pgm",'rb')
c_noi = open(".\Picture\Chess_noise.pgm",'rb')
chess_original = read_pgm(c_ori)
chess_noise = read_pgm(c_noi)
c_ori.close()
c_noi.close()

In [111]:
print("original :",RMS(chess_original,chess_original))
print("noise :",RMS(chess_original,chess_noise))
print("ideal 20 :",RMS(chess_original,chess_ideal_20))
print("ideal 30 :",RMS(chess_original,chess_ideal_30))
print("ideal 50 :",RMS(chess_original,chess_ideal_50))


original : 0.0
noise : 24.58393122921184
ideal 20 : 26.14214275049343
ideal 30 : 19.45855362928154
ideal 50 : 16.8634050088865


In [112]:
print("original :",RMS(chess_original,chess_original))
print("noise :",RMS(chess_original,chess_noise))
print("gaussian 20 :",RMS(chess_original,chess_gaussian_20))
print("gaussian 30 :",RMS(chess_original,chess_gaussian_30))
print("gaussian 50 :",RMS(chess_original,chess_gaussian_50))

original : 0.0
noise : 24.58393122921184
gaussian 20 : 20.73350018224493
gaussian 30 : 17.034469925094776
gaussian 50 : 14.096275506282188


In [114]:
print("original :",RMS(chess_original,chess_original))
print("noise :",RMS(chess_original,chess_noise))
print("median :",RMS(chess_original,chess_median_filter_non_pad))

original : 0.0
noise : 24.58393122921184
median : 23.66545750211011


Part save

Lenna

In [91]:
save_dir = open("./Picture/lenna_ideal_lowpass_20.pgm","wb")
writePGM(save_dir , len(lenna_ideal_contrast_20[0]) , len(lenna_ideal_contrast_20) , 255 , flatten(lenna_ideal_contrast_20))

In [92]:
save_dir = open("./Picture/lenna_ideal_lowpass_30.pgm","wb")
writePGM(save_dir , len(lenna_ideal_contrast_30[0]) , len(lenna_ideal_contrast_30) , 255 , flatten(lenna_ideal_contrast_30))

In [93]:
save_dir = open("./Picture/lenna_ideal_lowpass_50.pgm","wb")
writePGM(save_dir , len(lenna_ideal_contrast_50[0]) , len(lenna_ideal_contrast_50) , 255 , flatten(lenna_ideal_contrast_50))

In [58]:
save_dir = open("./Picture/lenna_Gaussian_lowpass_20.pgm","wb")
writePGM(save_dir , len(lenna_gaussian_contrast_20[0]) , len(lenna_gaussian_contrast_20) , 255 , flatten(lenna_gaussian_contrast_20))

In [59]:
save_dir = open("./Picture/lenna_Gaussian_lowpass_30.pgm","wb")
writePGM(save_dir , len(lenna_gaussian_contrast_30[0]) , len(lenna_gaussian_contrast_30) , 255 , flatten(lenna_gaussian_contrast_30))

In [60]:
save_dir = open("./Picture/lenna_Gaussian_lowpass_50.pgm","wb")
writePGM(save_dir , len(lenna_gaussian_contrast_50[0]) , len(lenna_gaussian_contrast_50) , 255 , flatten(lenna_gaussian_contrast_50))

In [80]:
save_dir = open("./Picture/lenna_median_lowpass.pgm","wb")
writePGM(save_dir , len(lenna_contrast[0]) , len(lenna_contrast) , 255 , flatten(lenna_contrast))

Chess

In [119]:
save_dir = open("./Picture/chess_ideal_lowpass_20.pgm","wb")
writePGM(save_dir , len(chess_ideal_contrast_20[0]) , len(chess_ideal_contrast_20) , 255 , flatten(chess_ideal_contrast_20))

In [120]:
save_dir = open("./Picture/chess_ideal_lowpass_30.pgm","wb")
writePGM(save_dir , len(chess_ideal_contrast_30[0]) , len(chess_ideal_contrast_30) , 255 , flatten(chess_ideal_contrast_30))

In [121]:
save_dir = open("./Picture/chess_ideal_lowpass_50.pgm","wb")
writePGM(save_dir , len(chess_ideal_contrast_50[0]) , len(chess_ideal_contrast_50) , 255 , flatten(chess_ideal_contrast_50))

In [61]:
save_dir = open("./Picture/chess_Gaussian_lowpass_20.pgm","wb")
writePGM(save_dir , len(chess_gaussian_contrast_20[0]) , len(chess_gaussian_contrast_20) , 255 , flatten(chess_gaussian_contrast_20))

In [62]:
save_dir = open("./Picture/chess_Gaussian_lowpass_30.pgm","wb")
writePGM(save_dir , len(chess_gaussian_contrast_30[0]) , len(chess_gaussian_contrast_30) , 255 , flatten(chess_gaussian_contrast_30))

In [63]:
save_dir = open("./Picture/chess_Gaussian_lowpass_50.pgm","wb")
writePGM(save_dir , len(chess_gaussian_contrast_50[0]) , len(chess_gaussian_contrast_50) , 255 , flatten(chess_gaussian_contrast_50))

In [100]:
save_dir = open("./Picture/chess_median_lowpass.pgm","wb")
writePGM(save_dir , len(chess_contrast[0]) , len(chess_contrast) , 255 , flatten(chess_contrast))